In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [18]:
df = hd.read_analysis_data("777")
hd.inspect(df)

Time for read_analysis_data            : 0.45 seconds
(689, 65)


,user_id,month,date,active_accounts,txns_count,txns_value,txn_count_sa,txn_count_ca,nunique_tag,nunique_tag_auto,nunique_merchant,prop_credit,month_income,year_income,has_regular_income,has_month_income,age,female,sa_inflows,has_sa_inflows,has_reg_sa_inflows,has_benefits,has_pension,has_mortgage_pmt,has_rent_pmt,loan_funds,loan_repmt,pdloan_funds,pdloan_repmt,region_name,is_urban,spend_finance,spend_services,spend_other_spend,spend_household,spend_motor,spend_communication,spend_retail,spend_travel,spend_hobbies,month_spend,entropy_tag_wsst,entropy_tag_ws,entropy_tag_wst,entropy_tag_w,entropy_tag_sst,entropy_tag_s,entropy_tag_st,entropy_tag,entropy_tag_auto_wsst,entropy_tag_auto_ws,entropy_tag_auto_wst,entropy_tag_auto_w,entropy_tag_auto_sst,entropy_tag_auto_s,entropy_tag_auto_st,entropy_tag_auto,entropy_merchant_wsst,entropy_merchant_ws,entropy_merchant_wst,entropy_merchant_w,entropy_merchant_sst,entropy_merchant_s,entropy_merchant_st,entropy_merchant
0,13777,11,2012-11-30,"[112648, 112650, 112652, 112651]",100,317126.781250,14.0,85.0,7,22,16,0.102756,6.36603,69.89982,0,1,43.0,0.0,50076.0,1,0,0,0,3.0,3.0,0,0,0,0,south east,0.0,1.15143,0.40366,1.686,6.983816,0.00000,0.0,0.50167,0.1369,0.02500,11.07382,-0.056132,3.633786,0.397855,3.109510,0.081098,2.623084,0.413394,2.363409,0.214615,8.01186,0.698136,4.333621,0.202806,6.895520,0.827278,3.884217,0.243232,10.720089,0.043733,3.861725,0.183947,9.748866,-0.076542,3.378306
1,13777,12,2012-12-31,"[112648, 112650, 112652, 112651, 112653]",65,34068.792969,8.0,52.0,7,18,15,0.022894,5.28394,69.89982,0,1,43.0,0.0,3250.0,1,0,0,0,3.0,1.0,0,0,0,0,south east,0.0,0.01353,0.14189,1.030,5.677650,0.20267,0.0,0.18395,0.0000,0.09499,7.34468,0.536063,3.778647,0.686832,3.305023,0.389008,2.705641,0.535191,2.430198,0.444588,8.04951,0.543808,4.189561,0.380756,6.946254,0.501481,3.606048,0.324177,10.722432,0.101888,3.922590,0.267102,9.754129,-0.082113,3.372841


In [19]:
dfe = df.filter(like="entropy")

In [20]:
dfe

,entropy_tag_wsst,entropy_tag_ws,entropy_tag_wst,entropy_tag_w,entropy_tag_sst,entropy_tag_s,entropy_tag_st,entropy_tag,entropy_tag_auto_wsst,entropy_tag_auto_ws,entropy_tag_auto_wst,entropy_tag_auto_w,entropy_tag_auto_sst,entropy_tag_auto_s,entropy_tag_auto_st,entropy_tag_auto,entropy_merchant_wsst,entropy_merchant_ws,entropy_merchant_wst,entropy_merchant_w,entropy_merchant_sst,entropy_merchant_s,entropy_merchant_st,entropy_merchant
0,-0.056132,3.633786,0.397855,3.109510,0.081098,2.623084,0.413394,2.363409,0.214615,8.011860,0.698136,4.333621,0.202806,6.895520,0.827278,3.884217,0.243232,10.720089,0.043733,3.861725,0.183947,9.748866,-0.076542,3.378306
1,0.536063,3.778647,0.686832,3.305023,0.389008,2.705641,0.535191,2.430198,0.444588,8.049510,0.543808,4.189561,0.380756,6.946254,0.501481,3.606048,0.324177,10.722432,0.101888,3.922590,0.267102,9.754129,-0.082113,3.372841
2,0.224038,3.702321,0.774260,3.364173,0.496779,2.734537,0.848588,2.602052,0.060730,7.986667,0.851267,4.476563,-0.038199,6.826808,0.759576,3.826413,-0.103243,10.710059,0.672352,4.519636,-0.171812,9.726349,0.570469,4.012942
3,0.637215,3.803391,1.059520,3.557172,1.064070,2.886639,1.228302,2.810272,-0.177075,7.947735,0.779944,4.409986,-0.152006,6.794362,0.885234,3.933700,-0.216277,10.706787,0.620377,4.465239,-0.236519,9.722254,0.576516,4.018873
4,0.588111,3.791379,1.040152,3.544068,0.938851,2.853065,1.161023,2.773379,-0.196089,7.944622,1.142062,4.748011,-0.226027,6.773258,1.171149,4.177817,-0.284870,10.704802,1.000041,4.862594,-0.320420,9.716943,0.946517,4.381799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,-0.580174,3.505597,0.431877,3.132529,-0.783620,2.391235,0.077761,2.179361,-0.987921,7.814988,0.583537,4.226647,-1.138690,6.513054,0.348070,3.475065,-0.697040,10.692871,1.043274,4.907841,-0.758995,9.689185,1.001454,4.435685
685,-0.252476,3.585757,0.560051,3.219247,-0.265429,2.530173,0.350935,2.329159,-0.588774,7.880334,0.784752,4.414474,-0.697935,6.638715,0.656367,3.738292,-0.676133,10.693476,0.799175,4.652368,-0.728915,9.691089,0.724097,4.163632
686,-0.779199,3.456911,0.174933,2.958689,-0.915949,2.355755,-0.098525,2.082693,-0.352280,7.919051,0.593074,4.235549,-0.494187,6.696804,0.436880,3.550892,0.006093,10.713224,1.058439,4.923713,0.036670,9.739545,1.191682,4.622274
687,-0.831708,3.444067,0.178408,2.961040,-0.798439,2.387262,-0.034911,2.117577,-0.762566,7.851881,0.227413,3.894217,-0.837358,6.598965,0.111744,3.273288,-0.448939,10.700053,0.703928,4.552683,-0.345461,9.715358,0.826817,4.264387
